In [1]:
import pandas as pd
import plotly.express as px

from index.GreenGrowthStuff import GreenGrowthStuff

In [2]:
data = (
    pd.read_csv('data/full_data/result.csv')
      .query("Aggregation == 'Indicator_normed' and Year == 2019")
    .query("ISO == 'FRA'")
)
info = GreenGrowthStuff().IND_CAT_DIM
data = data.merge(info, left_on='Variable', right_on='Indicator')

In [33]:

def Indicator_lolipop(ISO):
    df = data[(data.Aggregation == 'Indicator_normed') & (data.Year == 2019) & (data.ISO == ISO)]
#     df = df.dropna().groupby(
#         ['Variable', 'Dimension']).mean().reset_index()
    df = df.round(2).sort_values(by=['Dimension', 'Variable'], ascending=False)

    fig = px.scatter(df,
                     y='Variable',
                     x='Value',
                     color='Dimension',
                     facet_col='Dimension',
                     facet_col_spacing=0.05,
                     #hover_name='Variable_name',
                     hover_data={'Value': True, 'Variable': False},
                     labels={'Variable': 'Indicator', 'Value': 'Score'},
                     color_discrete_map={
                         "Social Inclusion": "#d9b5c9",
                         "Natural Capital Protection": "#f7be49",
                         "Efficient and Sustainable Resource Use": "#8fd1e7",
                         "Green Economic Opportunities": "#9dcc93",
                     },

                     height=600,
                     )
    fig.update_xaxes(showgrid=True, range=[0, 110], tickvals=[
                     20, 40, 60, 80, 100], visible=True, title='')
    fig.update_yaxes(showgrid=False, range=[-1, 12], matches=None, showticklabels=True)

    fig.update_traces(marker=dict(size=12, opacity=0.8))
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    bars = px.bar(df,
                  y='Variable',
                  x='Value',
                  facet_col='Dimension',
                  facet_col_spacing=0.05,
                  #hover_name='Variable_name',
                  hover_data={'Value': True},
                  labels={'Variable': 'Indicator', 'Value': ''},
                  orientation='h',
                  opacity=0.6,
                  height=600,
                  )

    bars.update_traces(marker_color='lightgrey',
                       width=0.1,
                       marker_line_width=0.1, opacity=0.8)

    fig.add_traces(bars.data)

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.1,
        xanchor="right",
        x=1,
        title=''
    ))
    return fig


In [34]:
category_lolipop(data)

In [ ]:
data = (
    pd.read_csv('data/full_data/result.csv')
      .query("Aggregation == 'Indicator_normed' and Year == 2019")
    .query("ISO == 'FRA'")
)
info = GreenGrowthStuff().IND_CAT_DIM
data = data.merge(info, left_on='Variable', right_on='Indicator')
data = pd.concat([data, pd.DataFrame({'Variable': ['GJ3', 'GT3', 'GV3', 'GN3'], 'Dimension': ['GEO'] * 4})]).sort_values(by=['Dimension', 'Variable'])

In [ ]:
data.Variable.values.reshape(16, -1)[0: 4]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def gauge_chart(data):
    #fig = go.Figure()
    n_indicator = data.shape[0]

    

    
    variables = data.Variable.values.reshape(16, -1)[8: 12].reshape(2, -1)
    values = data.Value.values.reshape(16, -1)[8: 12].reshape(2, -1)
        
    rows, cols = values.shape[1], values.shape[0]

    fig = make_subplots(rows=rows, cols=cols, )

    
    for i, (val_pair, var_pair) in enumerate(zip(values, variables)):
        
        for j in range(len(val_pair)):
            
            fig.add_trace(go.Indicator(
                value = round(val_pair[j], 0),
                gauge = {
                    'shape': "bullet",
                    'bgcolor': 'lightgrey',
                    'axis' : {'visible': False}
                },
                domain = {'row': j, 'column': i},
                title={"text": f"<br><span style='font-size:0.8em;color:gray'>{var_pair[j]}</span><br>"},
                          ))

        
    fig.update_layout(
        grid = {'rows': rows, 'columns': cols, 'xgap': 0.1, 'ygap': 0.3},
        template = {'data' : {'indicator': [{
                              'mode' : "number+gauge"
                                }]
                             }},
    height=400, width=1000,
)
    
    fig.update_traces(gauge_bar_thickness=1, gauge_axis_range=[0, 100], selector=dict(type='indicator'))
    return fig

In [ ]:
gauge_chart(data)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def gauge_chart(data):
    n_indicator = data.shape[0]

    fig = make_subplots(rows=n_indicator//2, cols=2, horizontal_spacing=1)
    

    
    variables = data.Variable.values
    values = data.Value.values

    
    
    for i, (val, var) in enumerate(zip(values, variables)):
        fig.add_trace(go.Indicator(
            value = val,
            gauge = {
                'shape': "bullet",
                'bgcolor': 'lightgrey',
                'axis' : {'visible': True}
            },
            domain = {'row': i, 'column': 0},
            title = {'text': var}),
                      )


        
    fig.update_layout(
        grid = {'rows': n_indicator, 'columns': 1, "ygap":0.5, "xgap": 0.5},
        template = {'data' : {'indicator': [{
                              'mode' : "number+gauge"
                                }]
                             }},
)
    
    fig.update_traces(gauge_bar_thickness=1, gauge_axis_range=[0, 100], selector=dict(type='indicator'))
    return fig

In [ ]:
gauge_chart(data.query('Variable in ["EE1", "EE2", "EE3", "EW1", "EW2", "EW3"]'))

In [ ]:
def get_missing_values_stat(data, max_year=2019):
    data = data[(data.Year >= 2005) & (data.Year <= max_year)]
    ind_cat_dim = GreenGrowthStuff().IND_CAT_DIM
    data = pd.merge(data, ind_cat_dim, on='Indicator')
    data['Year'] = data['Year'].astype(int)
    
    agg = ['ISO', 'Dimension']
    df = data.groupby(agg).apply(lambda x: x['Imputed'].sum() / x.shape[0] * 100)
    df = pd.DataFrame(df, columns=['% of imputed data'])
    df['imputed data points'] = data.groupby(
        agg).apply(lambda x: x['Imputed'].sum())

    df['corrected data points'] = data.groupby(
        agg).apply(lambda x: x['Corrected'].sum())

    df['% of corrected data points'] = data.groupby(agg).apply(
        lambda x: x['Corrected'].sum() / x.shape[0] * 100)

    df['Total data points'] = data.groupby(agg).apply(lambda x: x.shape[0])
    return df

In [ ]:
missing_data = get_missing_values_stat(data)#.loc['THA']

In [ ]:
import plotly.express as px

In [ ]:
plot_df = missing_val_stats.loc['THA'].reset_index()
plot_df = plot_df.melt(id_vars=['Dimension'], value_name='count').query('variable in ["Total data points", "imputed data points", "corrected data points"]')

In [ ]:
def missing_bar_plot(ISO):
    plot_df = missing_data.loc[ISO].reset_index()
    plot_df['s'] = plot_df['Dimension'].replace({'SI': 0, 'NCP': 1, 'ESRU': 2, "GEO": 3})
    plot_df =plot_df.sort_values(by='s')
    plot_df = plot_df.melt(id_vars=['Dimension'], value_name='count').query('variable in ["Total data points", "imputed data points", "corrected data points"]')
    fig = px.bar(plot_df,
             x='Dimension',
             y='count',
             color='variable',
             barmode='group',
             color_discrete_map={
                         "corrected data points": "#86BBD8",
                         "imputed data points": "#D8A488",
                         "Total data points": "#2db29b",
    })
    return fig

In [ ]:
missing_bar_plot('DEU')